In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'second-hand-car-price-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4869494%2F8215633%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240625%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240625T141741Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D132190da7248cdf3b96bb960c20bcf9fef0f166c8b960fc738ecf56b59d71da4cebbdb1dc5e068b47275d9828c01909b5b80835b3e0ef23a387bdd204f8c10e9af0da6e31473cedcad0c127c2a2edca515dbdd8d98b387ca1a7a1c35a58b95b1c21d34c2058bc80f1d1c9b6d0cdb646c06ed13f91c2ef9f0c9143132a93ec3dc6a8e93b06e8d1047e758e9bf84c2c81b5d28f23db173879c2cd486cae66bd609a90b113461a108c902be85d9604999201b3c7713f95ea8a5853a83048ff850ad287a29d4998413d75c3f67a8272271029231fb8f26228605711f78f2a608c73c2832f1657c6d63d15890a8bfb08dc5aed5764dd9e81dc23eeffa281da9739818'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load Data


In [ ]:
df = pd.read_csv('/kaggle/input/second-hand-car-price-prediction/cars.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
y = df['Price']
df.drop("Price", axis = 1, inplace = True)

In [ ]:
df.Year = df.Year.astype(str)
obj_cols = ["Year","Brand", "Model", "Fuel_Type", "Transmission", "Owner_Type"]
dummies = pd.get_dummies(df[obj_cols], drop_first=True, dtype='int')
dummies

In [ ]:
df.drop(obj_cols, inplace=True, axis=1)
df = pd.concat([df, dummies], axis=1)
df.head()

In [ ]:
df.drop('Car_ID', inplace=True, axis =1)

In [ ]:
X = df

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

print(len(X_train), len(y_train),len(X_test), len(y_test))

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)


In [ ]:
model.score(X_train, y_train)


In [ ]:
model.score(X_test, y_test)

## RMSE : Root MEan Squared Error

In [ ]:
predictions = model.predict(X_test)  # y-hat

In [ ]:
y_test # y

In [ ]:
y_test.values

In [ ]:
predictions

In [ ]:
import numpy as np
import math

np.sqrt(((y_test.values-predictions)**2).sum()/len(y_test))